In [1]:
library(data.table)
library(ggplot2)
# library(hrbrthemes)
library(reshape2)
library(tidyverse)
library(stringr)
library(heatmaply)
library(ape)
library(phylogram)
library(dendextend)
library(ComplexHeatmap)
library(RColorBrewer)
library(circlize)
theme_set(theme_bw())



Attaching package: ‘reshape2’


The following objects are masked from ‘package:data.table’:

    dcast, melt


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ tibble  3.1.6     ✔ dplyr   1.0.8
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1
✔ purrr   0.3.4     

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::between()   masks data.table::between()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::first()     masks data.table::first()
✖ dplyr::lag()       masks stats::lag()
✖ dplyr::last()      masks data.table::last()
✖ purrr::transpose() masks data.table::transpose()

Loading required package: plotly


Attaching package: ‘plotly’


The following object is masked from ‘package:ggplot2’:

    last_plot


The following object is masked from ‘package:stats’:

    filter


The following object is masked from ‘package:graphics’:

    layout


Loading required package: viridis

Loading r

ERROR: Error in library(ComplexHeatmap): there is no package called ‘ComplexHeatmap’


In [2]:

install.packages('ComplexHeatmap')
install.packages('circlize')

Warning message:
“package ‘ComplexHeatmap’ is not available for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [ ]:
get_ani <- function(
    path="/lustre/scratch/microbiome/n10853499/02-lorikeet_testing/02-mflorens_variant_test/03-lorikeet_out/GCA_003139855.1_20110800_S2M_genomic/", 
    full_metadata=full_metadata, 
    coverages
) {
    """
    Takes full path to lorikeet output folder for given genome
    full_metadata is generated below, and coverages should be read in automatically.
    You will just need to change the file paths to the corresponding files for you
    """
    
    suffix = tstrsplit(path, "/", keep=10L)
    pop_ani <- fread(paste0(path, "/", suffix, "_population_ani.tsv"), skip="SampleID",)
    con_ani <- fread(paste0(path, "/", suffix, "_consensus_ani.tsv"), skip="SampleID",)
    sub_pop_ani <- fread(paste0(path, "/", suffix, "_subpopulation_ani.tsv"), skip="SampleID",)
    # sample_labels
    con_ani[, SampleID:=factor(SampleID)]
    joined_con_ani <- full_join(con_ani, full_metadata, by=c("SampleID"))
    joined_con_ani <- full_join(joined_con_ani, coverages, by=c("run"="variable"))
    joined_con_ani[, sample_name_fixed := ifelse(is.na(sample_name), V1, sample_name)]
    joined_con_ani[, log10_abundance:=scale(value)]
    
    pop_ani[, SampleID:=factor(SampleID)]
    joined_pop_ani <- full_join(pop_ani, full_metadata, by=c("SampleID"))
    joined_pop_ani <- full_join(joined_pop_ani, coverages, by=c("run"="variable"))
    joined_pop_ani[, sample_name_fixed := ifelse(is.na(sample_name), V1, sample_name)]
    joined_pop_ani[, log10_abundance:=log10(value)]
    
    sub_pop_ani[, SampleID:=factor(SampleID)]
    joined_sub_pop_ani <- full_join(sub_pop_ani, full_metadata, by=c("SampleID"))
    joined_sub_pop_ani <- full_join(joined_sub_pop_ani, coverages, by=c("run"="variable"))
    joined_sub_pop_ani[, sample_name_fixed := ifelse(is.na(sample_name), V1, sample_name)]
    joined_sub_pop_ani[, log10_abundance:=log10(value)]
    
    return(list(con=joined_con_ani, pop=joined_pop_ani, sub_pop=joined_sub_pop_ani))
}

In [ ]:
header <- fread("/mnt/hpccs01/scratch/microbiome/n10853499/02-lorikeet_testing/02-mflorens_variant_test/03-lorikeet_out/GCA_003139855.1_20110800_S2M_genomic/GCA_003139855.1_20110800_S2M_genomic_population_ani.tsv")
metadata <- fread("/mnt/hpccs01/scratch/microbiome/n10853499/02-lorikeet_testing/02-mflorens_variant_test/sample_annotations.txt")[run != "-----------"]
metadata[, location:=ifelse(grepl("\\.NA", sample_name, fixed=F) | startsWith(sample_name, "20120700"), "Abisko", 
                                  ifelse(startsWith(sample_name, "G") & taxon_name=="peat metagenome" | startsWith(sample_name, "MG_"), "Marcell", 
                                     ifelse(startsWith(sample_name, "G"), "Loxahatchee", 
                                            ifelse(sample_name=="Metagenome", "Chersky", 
                                                   ifelse(grepl("AK5YR", sample_name, fixed=F), "Alaska", 
                                                          ifelse(startsWith(sample_name, "S1071") | startsWith(sample_name, "BPON"), "Alaska", 
                                                                 ifelse(model=="NextSeq 500", "Abisko", 
                                                                        "Other")))))))]
metadata[, taxon_name:=str_replace(taxon_name, " metagenome", "")]
metadata[, taxon_name:=str_replace(taxon_name, "sediment", "sdmnt.")]
metadata[, type:=ifelse(location=='Abisko', ifelse(
    grepl('_E', sample_name, fixed=F), 'Fen', ifelse(grepl('_S', sample_name, fixed=F), 'Bog', ifelse(grepl('_P', sample_name, fixed=F), 'Palsa', NA))), NA)]
header[, V1:=str_replace(V1, "##sample=<ID=", "")]
header[, run:=str_replace(`##source=lorikeet-v0.6.5`, "name=04-bams/coverm-genome\\.", "")]
header[, `##source=lorikeet-v0.6.5`:=NULL]
header[, run:=str_replace(run, "_1\\.fastq\\.gz\\.bam>", "")]
header[, run:=str_replace(run, "_1\\.fastq\\.bam>", "")]

# header$`##source=lorikeet-v0.6.1` = NULL
full_metadata <- full_join(metadata, header, by.x="run", by.y="run")
mflorens_coverage <- fread("//mnt/hpccs01//scratch/microbiome/n10853499/02-lorikeet_testing/02-mflorens_variant_test/bog_coverage_and_abundances.tsv")

mflorens_coverage_m <- as.data.table(melt(mflorens_coverage))
mflorens_coverage_m <- mflorens_coverage_m[grepl("Trimmed", variable)]
mflorens_coverage_m[, variable:=gsub("_1..*", "", variable)]
mflorens_coverage_m[, variable:=gsub("coverm-genome\\.", "", variable)]


full_metadata[, SampleID:=factor(V1)]

bog38 <- get_ani("/mnt/hpccs01/scratch/microbiome/n10853499/02-lorikeet_testing/02-mflorens_variant_test/03-lorikeet_out/GCA_003139855.1_20110800_S2M_genomic/", full_metadata=full_metadata, coverage=mflorens_coverage_m[Genome=="GCA_003139855.1_20110800_S2M_genomic"])
fen3 <- get_ani("/mnt/hpccs01/scratch/microbiome/n10853499/02-lorikeet_testing/02-mflorens_variant_test/03-lorikeet_out/GCA_003162175.1_20100900_E2D_genomic/", full_metadata=full_metadata, coverage=mflorens_coverage_m[Genome=="GCA_003162175.1_20100900_E2D_genomic"])
fen53 <- get_ani("/mnt/hpccs01/scratch/microbiome/n10853499/02-lorikeet_testing/02-mflorens_variant_test/03-lorikeet_out/GCA_003158275.1_20120600_E3D_genomic/", full_metadata=full_metadata, coverage=mflorens_coverage_m[Genome=="GCA_003158275.1_20120600_E3D_genomic"])

In [ ]:
generate_heatmap <- function(input_table, to_select=seq(1,208)) {
    """
    Function for generating annotated ANI heatmaps
    input table is the table generated by get_ani
    to_select represents the range of samples to select from
    """
    # Consensus heatmap
    min_ani <- min(input_table$con[to_select, to_select + 1, with=FALSE], input_table$pop[to_select, to_select + 1, with=FALSE], input_table$sub_pop[to_select, to_select + 1, with=FALSE])
    col_fun <- colorRamp2(c(min_ani, (1 + min_ani) / 2, 1), c("#2c7fb8", "#7fcdbb", "#edf8b1"))
    abisko_colours <- setNames(c("#0001FF", "#058000", "#703C1B"), c("Fen", "Bog", "Palsa"))
#     abisko_colours <- list("Fen"="#0001FF", "Bog"="#058000", "Palsa"="#703C1B")
    max_ref_ani_diff <- 1 - min(
        diag(as.matrix(input_table$con[to_select, to_select + 1, with=FALSE])), 
        diag(as.matrix(input_table$pop[to_select, to_select + 1, with=FALSE])), 
        diag(as.matrix(input_table$sub_pop[to_select, to_select + 1, with=FALSE]))
    )
    con <- as.matrix(input_table$con[to_select, to_select + 1, with=FALSE])
    
    #ani difference from reference
    con_ref_diff <- diag(con)
    # set to NA
    diag(con) <- NA
    
    rownames(con) <- input_table$con$SampleId
    colnames(con) <- input_table$con$SampleId
    set1_con <- RColorBrewer::brewer.pal(name="Set3", n=9)
    names(set1_con) <- sort(unique(input_table$con$taxon_name))
    set2_con <- RColorBrewer::brewer.pal(name="Set3", n=6)
    names(set2_con) <- sort(unique(input_table$con$location))
    
    top_ha_con <- HeatmapAnnotation(
        ` `=anno_barplot(1 - con_ref_diff,
        ylim=c(0, max_ref_ani_diff))
    )
    bot_ha_con <- HeatmapAnnotation(
                                 ` `=as.factor(input_table$con$location[to_select]), 
                                col=list(
                                    ` `=set2_con
                                    ), 
                                    show_legend = c(FALSE)
                                )
    row_loc <- rowAnnotation(
                                 `Cov.`=anno_barplot(input_table$con$value[to_select]),
                                 `Loc.`=as.factor(input_table$con$location[to_select]),
                                 `Abisko`=as.factor(input_table$con$type[to_select]),
                                col=list(
                                    `Loc.`=set2_con,
                                    `Abisko`=abisko_colours
                                    ), show_legend = c(FALSE, FALSE, TRUE),
                                annotation_name_rot = 90
                                )

    ha_con <- Heatmap(con, name="ANI", bottom_annotation=bot_ha_con, top_annotation=top_ha_con, left_annotation=row_loc, column_title="conANI", col=col_fun)
    
    # population
    pop <- as.matrix(input_table$pop[to_select, to_select + 1, with=FALSE])
    rownames(pop) <- input_table$pop$SampleId
    colnames(pop) <- input_table$pop$SampleId
    set1_pop <- RColorBrewer::brewer.pal(name="Set3", n=9)
    names(set1_pop) <- sort(unique(input_table$pop$taxon_name))
    set2_pop <- RColorBrewer::brewer.pal(name="Set3", n=6)
    names(set2_pop) <- sort(unique(input_table$pop$location))
    
    # ani difference from reference
    pop_ref_diff <- diag(pop)
    # set to NA
    diag(pop) <- NA
    
    top_ha_pop <- HeatmapAnnotation(` `=anno_barplot(1-pop_ref_diff,
        ylim=c(0, max_ref_ani_diff)))
    bot_ha_pop <- HeatmapAnnotation(` `=as.factor(input_table$pop$location[to_select]), # Variable names are just different number of spaces lol. To prevent text on concatenated plot

                                col=list(
                                    ` `=set2_pop
                                    ), 
                                    show_legend = c(FALSE)
                                )

    ha_pop <- Heatmap(pop, name="ANI", bottom_annotation=bot_ha_pop, top_annotation=top_ha_pop, column_title="popANI", col=col_fun)
    
    # sub population
    sub_pop <- as.matrix(input_table$sub_pop[to_select, to_select + 1, with=FALSE])
    rownames(sub_pop) <- input_table$sub_pop$SampleId
    colnames(sub_pop) <- input_table$sub_pop$SampleId
    set1_sub_pop <- RColorBrewer::brewer.pal(name="Set3", n=9)
    names(set1_sub_pop) <- sort(unique(input_table$sub_pop$taxon_name))
    set2_sub_pop <- RColorBrewer::brewer.pal(name="Set3", n=6)
    names(set2_sub_pop) <- sort(unique(input_table$sub_pop$location))
    
    # ani difference from reference
    sub_pop_ref_diff <- diag(sub_pop)
    # set to NA
    diag(sub_pop) <- NA
    
    top_ha_sub_pop <- HeatmapAnnotation(`Ref. Diff`=anno_barplot(1-sub_pop_ref_diff,
        ylim=c(0, max_ref_ani_diff)))
    bot_ha_sub_pop <- HeatmapAnnotation(`Location`=as.factor(input_table$sub_pop$location[to_select]), 
                                        col=list(
                                            `Location`=set2_sub_pop
                                            ), 
                                        show_legend = c(TRUE, TRUE),
                                        annotation_legend_param = list(
                                            `Location` = list(direction = "horizontal")
                                        ))

    ha_sub_pop <- Heatmap(sub_pop, name="ANI", bottom_annotation=bot_ha_sub_pop, top_annotation=top_ha_sub_pop, column_title="subpopANI", col=col_fun)
    ht_list = ha_con + ha_pop + ha_sub_pop
    return(ht_list)
}


# These cells print out the ANI plots to current directory

In [ ]:
bog38_list <- generate_heatmap(bog38)
# bog38_list
png("bog38_ani_maps_wide.png",width=20,height=9,units="in",res=1200)
draw(bog38_list, merge_legend=T, column_title="GCA_003139855.1 ANI distributions (Bog38)", heatmap_legend_side = "right", annotation_legend_side = "bottom")
dev.off()